In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [2]:
spark = SparkSession.builder.master('local').appName('Test').getOrCreate()

23/09/03 10:38:28 WARN Utils: Your hostname, MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 172.20.10.2 instead (on interface en0)
23/09/03 10:38:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/03 10:38:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<b>Task 1</b>

In [3]:
path = "/Users/Savelyev_Ilya/Desktop/Task1.csv"

In [4]:
df = spark.read.option("header", True).csv(path)

In [5]:
df.show()

+----------+----------+----------+-----------------+
|id_cliente|    nombre|       dia|numero de accesos|
+----------+----------+----------+-----------------+
|         1|    Repsol|01.01.2023|               30|
|         2|Telefonica|01.01.2023|               10|
|         1|      null|02.01.2023|               20|
|         1|      null|03.01.2023|                5|
|         2|      null|03.01.2023|                7|
|         3|   Inditex|04.01.2023|                4|
+----------+----------+----------+-----------------+



Create new dataframe with distinct names

In [6]:
df1 = df.filter(f.col('nombre').isNotNull()).distinct().orderBy('id_cliente').drop('dia', 'numero de accesos')

In [7]:
df1.show()

+----------+----------+
|id_cliente|    nombre|
+----------+----------+
|         1|    Repsol|
|         2|Telefonica|
|         3|   Inditex|
+----------+----------+



Join two dataframes, drop the old names column and leave the new one

In [13]:
new_df = df.drop('nombre').join(df1, how = 'left_outer', on = ["id_cliente"])
new_df = new_df.select('id_cliente', 'nombre', 'dia', 'numero de accesos')

In [14]:
new_df.show()

+----------+----------+----------+-----------------+
|id_cliente|    nombre|       dia|numero de accesos|
+----------+----------+----------+-----------------+
|         1|    Repsol|01.01.2023|               30|
|         2|Telefonica|01.01.2023|               10|
|         1|    Repsol|02.01.2023|               20|
|         1|    Repsol|03.01.2023|                5|
|         2|Telefonica|03.01.2023|                7|
|         3|   Inditex|04.01.2023|                4|
+----------+----------+----------+-----------------+



<b>Task 2</b>

In [15]:
path = "/Users/Savelyev_Ilya/Desktop/Task2.csv"

In [16]:
df2 = spark.read.option("delimiter", ";").option("header", True).csv(path)

In [17]:
df2.show()

+----------+-----------+----------+-----------------+
|id cliente|     nombre|       dia|numero de accesos|
+----------+-----------+----------+-----------------+
|         1|     Repsol|01.01.2023|               30|
|         1|       null|02.01.2023|               20|
|         1|Repsol S.A.|03.01.2023|                5|
|         1|       null|04.01.2023|               10|
+----------+-----------+----------+-----------------+



Create new dataframe with last used name

In [30]:
df2_name = df2.filter(f.col('nombre').isNotNull()).orderBy(f.col('dia').desc()).limit(1)
df2_name = df2_name.drop('dia', 'numero de accesos')

In [31]:
df2_name.show()

+----------+-----------+
|id cliente|     nombre|
+----------+-----------+
|         1|Repsol S.A.|
+----------+-----------+



Join two dataframes, drop the old names column and apply last used name to all the rows

In [32]:
df2_new = df2.drop('nombre').join(df2_name, how = 'left_outer', on = ['id cliente'])
df2_new = df2_new.select('id cliente', 'nombre', 'dia', 'numero de accesos')

In [33]:
df2_new.show()

+----------+-----------+----------+-----------------+
|id cliente|     nombre|       dia|numero de accesos|
+----------+-----------+----------+-----------------+
|         1|Repsol S.A.|01.01.2023|               30|
|         1|Repsol S.A.|02.01.2023|               20|
|         1|Repsol S.A.|03.01.2023|                5|
|         1|Repsol S.A.|04.01.2023|               10|
+----------+-----------+----------+-----------------+



<b>Task 3</b>

In [40]:
path = "/Users/Savelyev_Ilya/Desktop/Task3_1.csv"

In [41]:
df3_1 = spark.read.option("delimiter", ";").option("header", True).csv(path)

In [42]:
df3_1.show()

+----------+----------+----------+-----------------+
|id cliente|    nombre|       dia|numero de accesos|
+----------+----------+----------+-----------------+
|         1|Repsol S.A|01.01.2023|               30|
|         1|      null|02.01.2023|               20|
|         1|Repsol S.A|02.01.2023|                5|
+----------+----------+----------+-----------------+



In [37]:
path = "/Users/Savelyev_Ilya/Desktop/Task3_2.csv"

In [38]:
df3_2 = spark.read.option("delimiter", ";").option("header", True).csv(path)

In [39]:
df3_2.show()

+----------+----------+----------+-----------------+
|id cliente|    nombre|       dia|numero de accesos|
+----------+----------+----------+-----------------+
|         1|Repsol S.A|01.01.2023|               30|
|         1|      null|02.01.2023|               25|
+----------+----------+----------+-----------------+



Create new dataframe with antijoin for the first dataframe

In [59]:
df3_1_anti = df3_1.withColumn('df', f.lit('df3_1')).join(df3_2, (df3_1['id cliente'] == df3_2['id cliente']) & (df3_1['nombre'] == df3_2['nombre']) & (df3_1['dia'] == df3_2['dia']) & (df3_1['numero de accesos'] == df3_2['numero de accesos']),'anti')

In [60]:
df3_anti.show()

+----------+----------+----------+-----------------+-----+
|id cliente|    nombre|       dia|numero de accesos|   df|
+----------+----------+----------+-----------------+-----+
|         1|      null|02.01.2023|               20|df3_1|
|         1|Repsol S.A|02.01.2023|                5|df3_1|
+----------+----------+----------+-----------------+-----+



Create new dataframe with antijoin for the second dataframe

In [61]:
df3_2_anti = df3_2.withColumn('df', f.lit('df3_2')).join(df3_1, (df3_1['id cliente'] == df3_2['id cliente']) & (df3_1['nombre'] == df3_2['nombre']) & (df3_1['dia'] == df3_2['dia']) & (df3_1['numero de accesos'] == df3_2['numero de accesos']),'anti')

In [62]:
df3_2_anti.show()

+----------+------+----------+-----------------+-----+
|id cliente|nombre|       dia|numero de accesos|   df|
+----------+------+----------+-----------------+-----+
|         1|  null|02.01.2023|               25|df3_2|
+----------+------+----------+-----------------+-----+



Union two new dataframes in one

In [73]:
df3_new = df3_1_anti.union(df3_2_anti).select('df', 'id cliente', 'nombre', 'dia', 'numero de accesos').orderBy(f.col('numero de accesos').desc())

In [74]:
df3_new.show()

+-----+----------+----------+----------+-----------------+
|   df|id cliente|    nombre|       dia|numero de accesos|
+-----+----------+----------+----------+-----------------+
|df3_1|         1|Repsol S.A|02.01.2023|                5|
|df3_2|         1|      null|02.01.2023|               25|
|df3_1|         1|      null|02.01.2023|               20|
+-----+----------+----------+----------+-----------------+



In [75]:
spark.stop()